2024.11.20
코드 만들다가, colormath가 사용하는 numpy가 1.16.0 version의 구형인데 업데이트가 안되어서 활용 못했음
그냥 엑셀 VBA code로 해결했다.

😎VBA code 

Function DegToRad(Deg As Double) As Double
    DegToRad = Deg * WorksheetFunction.Pi() / 180
End Function

Function RadToDeg(Rad As Double) As Double
    RadToDeg = Rad * 180 / WorksheetFunction.Pi()
End Function

Function CIEDE2000(L1 As Double, a1 As Double, b1 As Double, L2 As Double, a2 As Double, b2 As Double) As Double
    Dim dL As Double
    Dim Lavg As Double
    Dim C1 As Double, C2 As Double, Cavg As Double
    Dim G As Double, a1Prime As Double, a2Prime As Double
    Dim C1Prime As Double, C2Prime As Double, CavgPrime As Double
    Dim dCPrime As Double, h1Prime As Double, h2Prime As Double
    Dim dhPrime As Double, dH As Double
    Dim T As Double, SL As Double, SC As Double, SH As Double, RT As Double

    ' Calculate differences and means
    Lavg = (L1 + L2) / 2
    dL = L2 - L1
    C1 = Sqr(a1 ^ 2 + b1 ^ 2)
    C2 = Sqr(a2 ^ 2 + b2 ^ 2)
    Cavg = (C1 + C2) / 2
    
    G = 0.5 * (1 - Sqr((Cavg ^ 7) / (Cavg ^ 7 + 25 ^ 7)))
    a1Prime = a1 * (1 + G)
    a2Prime = a2 * (1 + G)
    C1Prime = Sqr(a1Prime ^ 2 + b1 ^ 2)
    C2Prime = Sqr(a2Prime ^ 2 + b2 ^ 2)
    CavgPrime = (C1Prime + C2Prime) / 2
    dCPrime = C2Prime - C1Prime
    
    If (b1 = 0 And a1Prime = 0) Then
        h1Prime = 0
    Else
        h1Prime = RadToDeg(Application.WorksheetFunction.Atan2(b1, a1Prime))
        If h1Prime < 0 Then h1Prime = h1Prime + 360
    End If
    
    If (b2 = 0 And a2Prime = 0) Then
        h2Prime = 0
    Else
        h2Prime = RadToDeg(Application.WorksheetFunction.Atan2(b2, a2Prime))
        If h2Prime < 0 Then h2Prime = h2Prime + 360
    End If

    If Abs(h1Prime - h2Prime) <= 180 Then
        dhPrime = h2Prime - h1Prime
    Else
        If h2Prime > h1Prime Then
            dhPrime = h2Prime - h1Prime - 360
        Else
            dhPrime = h2Prime - h1Prime + 360
        End If
    End If
    
    dH = 2 * Sqr(C1Prime * C2Prime) * Sin(DegToRad(dhPrime) / 2)
    
    T = 1 - 0.17 * Cos(DegToRad((h1Prime + h2Prime) / 2 - 30)) + 0.24 * Cos(DegToRad(2 * (h1Prime + h2Prime) / 2)) _
        + 0.32 * Cos(DegToRad(3 * (h1Prime + h2Prime) / 2 + 6)) - 0.20 * Cos(DegToRad(4 * (h1Prime + h2Prime) / 2 - 63))
    
    SL = 1 + 0.015 * (Lavg - 50) ^ 2 / Sqr(20 + (Lavg - 50) ^ 2)
    SC = 1 + 0.045 * CavgPrime
    SH = 1 + 0.015 * CavgPrime * T
    
    RT = -2 * Sqr((CavgPrime ^ 7) / (CavgPrime ^ 7 + 25 ^ 7)) * Sin(DegToRad(60 * Exp(-((h1Prime + h2Prime) / 2 - 275) / 25) ^ 2))
    
    CIEDE2000 = Sqr((dL / SL) ^ 2 + (dCPrime / SC) ^ 2 + (dH / SH) ^ 2 + RT * (dCPrime / SC) * (dH / SH))
End Function

In [1]:
import time
import os
import pandas as pd  
import re
import seaborn as sns
import matplotlib.pyplot as plt 
import openpyxl
import math

In [2]:
os.chdir("/Users/ysoh/Desktop/FA팀/3.맞춤형쿠션/2024 조색 샘플 측색")
pd.options.display.float_format='{:.2f}'.format
df = pd.read_csv('./20241120_deltae.csv', encoding='cp949')

In [3]:
!pip install colormath
from colormath import color_diff_matrix
from colormath.color_diff import delta_e_cie2000
from colormath.color_objects import LabColor

In [4]:
import skimage
from skimage import color

In [5]:
df

,Name,L(m),a(m),b(m),C(m),h°(m),L(g),a(g),b(g),C(g),h°(g)
0,G1C3,84.71,7.22,14.20,15.93,63.04,85.02,7.15,13.85,15.59,62.71
1,G1C2,84.84,8.55,16.12,18.24,62.05,85.89,7.47,15.93,17.59,64.87
2,G1C1,83.28,8.08,15.11,17.14,61.85,84.73,6.95,14.38,15.97,64.22
3,G1N,85.39,7.16,16.18,17.69,66.12,85.47,7.25,16.17,17.72,65.85
4,G1W1,85.44,6.35,16.38,17.57,68.81,85.48,6.17,15.75,16.92,68.63
...,...,...,...,...,...,...,...,...,...,...,...
252,G48W,25.67,7.43,14.17,16.00,62.32,25.31,7.37,14.25,16.04,62.64
253,G49C,20.00,9.86,11.95,15.49,50.49,19.15,9.46,12.31,15.53,52.47
254,G49N,21.07,7.89,11.49,13.94,55.50,19.48,7.30,10.89,13.11,56.16
255,G49W,22.48,5.57,11.11,12.43,63.36,21.25,5.32,10.49,11.76,63.08


In [4]:
import numpy as np

In [12]:
pip install --upgrade colormath

Note: you may need to restart the kernel to use updated packages.


In [ ]:
s1= {'Name':['G1C3'], 'L(m)':[54.34], 'C(m)':[44.18], 'h°(m)':[43.82], 'a(m)':[31.87], 'b(m)':[30.59]}
df1=pd.DataFrame(s1)

sample1_index=df1['Name'][0]
sample1_L=df1['L(m)'][0]
sample1_a=df1['a(m)'][0]
sample1_b=df1['b(m)'][0]
sample1_C=df1['C(m)'][0]
sample1_h=df1['h°(m)'][0]

sample2_index=[]
sample2_L=[]
sample2_a=[]
sample2_b=[]
sample2_C=[]
sample2_h=[]

delta_e=[]


i=0
for i in range(len(df)):
  s2 = df.iloc[i, 0:6]

  e1 = LabColor(df1['L(m)'], df1['a(m)'], df1['b(m)'])
  e2 = LabColor(s2['L(m)'], s2['a(m)'], s2['b(m)'])
    
  sample1_index=df1['Name'][0]
  sample1_L=df1['L(m)'][0] 
  sample1_a=df1['a(m)'][0]
  sample1_b=df1['b(m)'][0] 
  sample1_C=df1['C(m)'][0]
  sample1_h=df1['h°(m)'][0]

  sample2_index.append(i)
  sample2_L.append(s2['L* '])
  sample2_a.append(s2['a* '])
  sample2_b.append(s2['b* '])
  sample2_C.append(s2['C* '])
  sample2_h.append(s2['h° '])

  delta_e_cie2000(e1,e2)
  delta_e.append(delta_e_cie2000(e1,e2))

  i=i+1


items = {'std1':sample1_index,'s2': pd.Series(data = sample2_index),'delta_e': pd.Series(data = delta_e),
         'std1_L':sample1_L,'s2_L': pd.Series(data = sample2_L),
         'std1_a':sample1_a,'s2_a': pd.Series(data = sample2_a),
         'std1_b':sample1_b,'s2_b': pd.Series(data = sample2_b),
         'std1_C':sample1_C,'s2_C': pd.Series(data = sample2_C),
         'std1_h°':sample1_h,'s2_h°': pd.Series(data = sample2_h)}

df_items = pd.DataFrame(items)
#df_items_1=df_items.drop_duplicates(df_items)

#delta_e 0인 행 제거
idx = df_items[df_items['delta_e']==0].index
df_items_2 = df_items.drop(idx)

#CSV파일로 저장
df_Red = pd.DataFrame(df_items_2, columns = ['std1','s2','delta_e','std1_L','s2_L','std1_a','s2_a','std1_b','s2_b','std1_C','s2_C','std1_h°','s2_h°'])
df_Red.to_csv('20241120_xrite_delaE.csv')

In [11]:
from colormath.color_conversions import convert_color
from colormath.color_objects import LabColor, AdobeRGBColor

def delta_e_cie2000_alternative(color1, color2):
    # Convert to LabColor objects if not already
    if not isinstance(color1, LabColor):
        color1 = convert_color(color1, LabColor)
    if not isinstance(color2, LabColor):
        color2 = convert_color(color2, LabColor)

    # Perform CIE2000 delta E calculation
    delta_e = delta_e_cie2000(color1, color2)

    return delta_e

In [12]:


for i in range(len(df)):
    #s1= {'Name':[df['Name'][i]], 'L(m)':[['L(m)'][i]], 'C(m)':[['C(m)'][i]], 'h°(m)':[['h°(m)'][i]], 'a(m)':[['a(m)'][i]], 'b(m)':[['b(m)'][i]]}
    #df1=pd.DataFrame(s1)
    
    # Initialize lists to store results
    sample2_index = []
    sample2_l = []
    sample2_a = []
    sample2_b = []
    sample2_c = []
    sample2_h = []
    delta_e = []
    
    # Create color objects for df
    s2_row = df.iloc[i]
    e1 = LabColor(lab_l=df['L(m)'][i], lab_a=df['a(m)'][i], lab_b=df['b(m)'][i])
    e2 = LabColor(lab_l=df['L(g)'][i], lab_a=df['a(g)'][i], lab_b=df['b(g)'][i])
    
    # Store information
    sample2_index.append(s2_row['Name'])
    sample2_l.append(s2_row['L(g)'])
    sample2_a.append(s2_row['a(g)'])
    sample2_b.append(s2_row['b(g)'])
    sample2_c.append(s2_row['C(g)'])
    sample2_h.append(s2_row['h°(g)'])
    
    # Calculate delta E
    delta_e_cie2000_alternative(e1,e2)
    delta_e.append(delta_e_cie2000(e1,e2))
    i=i+1
    

AttributeError: module 'numpy' has no attribute 'asscalar'

In [13]:
color_diff.py

NameError: name 'color_diff' is not defined

In [ ]:
탭을 젖히면서 생각했다. '이 흑맥주보다 그 때 마신 맥주가 훨씬 좋았는데.'
오래 전 혹한의 1월 밤인가, 코젤 다크 하우스에 처음 다녀온 뒤 그는 이전까지 G사의 흑맥주에 의해 훼손된 감상을 새로 썼다. 
워낙에 알코올 분해 대사가 잘 안되는 간을 가져서 그가 편하게 술을 즐길 수 있는 주류는 몇 가지 되지 않았다.  
그래서인지 그는 많은 주종 가운데서도 맥주를, 특히 풍부한 향미와 묵직한 바디감을 가진 흑맥주를 좋아했다. 한 모금 머금는 것만으로도 진한 인상을 주는 술.
목 넘김이 뜨겁다거나 혓바닥이 쓰라려서 몸서리치게 하지 않는 술. 그의 신체는 삶의 고매함을 좇는 허영심에 비하면 크게 허술하고 너절했으며, 연약했다. 
첫 잔에 오크통 명찰을 잘못 보고 가져온 후진 맥주를 대충 비우고 온 터라, 둘쨋번은 맛이 좋을 줄 알았다. 이상하네...같은 코젤 다크 비어인데 왜 이렇게 맛이 없지.
...설마, 나 흑설탕을 좋아하는 건가. 맥주가 문제가 아니고 흑설탕이 필요한건가. 큰일이다. 술도 못 마시는데 단 맛 없으면 술 맛도 모르는 사람이란 말인가. 
그는 잠시 혼자만의 자괴감에 빠져들다가 필사적으로 도망쳤다. 아니다, 생맥주가 아니라서 그렇다. 그래, 바로 그 점이 큰 차이점이다. 그렇다면 이걸 확실하게
확인하기 위해서는 코젤 다크 하우스에 가야만 한다. 가서 프로스팅 빼고 그냥 맥주만 바로 뽑아서 달라고 하는 것이다. 그는 기세등등하게 결론을 내렸다.
그러는 동안에도 탭에서는 126cc, 128cc, ...1213cc 계속해서 맥주가 쏟아졌다. 뒤에서 수군거렸다. "여기와서 저렇게 마실 거면 차라리 코젤 다크하우스를 가지."